### Makine Öğrenimi ile Cinsiyet Tespiti

In [5]:
import cv2
import cvlib as cv
import sys
import numpy as np
from numpy.lib.type_check import imag
image = cv2.imread("girl2.jpg")

In [6]:
face, confidence = cv.detect_face(image)
padding = 20

for i in face:
    (x, y) = max(0, i[0]-padding), max(0, i[1]-padding)
    (x2, y2) = min(image.shape[1]-1, i[2]+padding), min(image.shape[0]-1,i[3]+padding)
    cv2.rectangle(image, (x, y), (x2, y2), (0, 255, 0), 2)
    crop = np.copy(image[y:y2, x:x2])
    (label, confidence) = cv.detect_gender(crop)
    idx = np.argmax(confidence)
    label = label[idx]
    label = "{}: {:.2f}%".format(label, confidence[idx] * 100)
    Y = y - 10 if y -10 > 10 else y + 10
    cv2.putText(image, label, (y, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2.imshow("Gender Detection", image)
cv2.waitKey()
cv2.destroyAllWindows()

Exception: URL fetch failure on https://download.cvlib.net/config/gender_detection/gender_deploy.prototxt: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)

### Python ile Yaş ve Cinsiyet Tespiti

In [20]:
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencv2Dnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencv2Dnn, bboxes

In [24]:
face = cv2.imread("face.jpg")

In [ ]:
# Cinsiyet Algılama

In [26]:
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
genderNet = cv2.dnn.readNet(genderModel, genderProto)

genderList = ['Male', 'Female']

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
genderNet.setInput(blob)
genderPreds = genderNet.forward()
gender = genderList[genderPreds[0].argmax()]
print("Gender Output : {}".format(genderPreds))
print("Gender : {}".format(gender))

Gender Output : [[5.4802080e-05 9.9994516e-01]]
Gender : Female


In [27]:
# Yaş Algılama

In [28]:
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
ageNet = cv2.dnn.readNet(ageModel, ageProto)

ageList = ['(0 - 2)', '(4 - 6)', '(8 - 12)', '(15 - 20)', '(25 - 32)', '(38 - 43)', '(48 - 53)', '(60 - 100)']

ageNet.setInput(blob)
agePreds = ageNet.forward()
age = ageList[agePreds[0].argmax()]
print("Gender Output : {}".format(agePreds))
print("Gender : {}".format(age))

Gender Output : [[3.1068844e-06 5.2632924e-05 4.3635042e-03 2.0448084e-03 9.3114686e-01
  3.1577752e-03 5.9003104e-02 2.2820616e-04]]
Gender : (25 - 32)


In [ ]:
# Herhangi bir resimden Yaş ve Cinsiyet

In [52]:
import cv2

# Resim dosyasını yükleyin
image = cv2.imread("Familyy.jpg") # tahmin ettirmek istediğimiz resmi tanımlıyoruz.
#image = cv2.imread("face2.jfif")

# Yaş ve cinsiyet tahmini için model ve prototip dosyalarını yükleyin
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
genderNet = cv2.dnn.readNet(genderModel, genderProto)

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
ageNet = cv2.dnn.readNet(ageModel, ageProto)

ageList = ['(0 - 2)', '(4 - 6)', '(8 - 12)', '(15 - 20)', '(25 - 32)', '(38 - 43)', '(48 - 53)', '(60 - 100)']
genderList = ['Male', 'Female']

# Görüntüyü gri tonlamalıya dönüştürün
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Yüz algılaması yapın
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

# Algılanan yüz bölgelerini işleyin
for (x, y, w, h) in faces:
    # Yüzü çerçeve üzerine çizin
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Yüz bölgesini yakalayın
    face = image[y:y + h, x:x + w]

    # Yüz bölgesini işlemek için boyutu yeniden ayarlayın (modelin beklediği boyut)
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

    # Cinsiyet tahmini yapın
    genderNet.setInput(blob)
    genderPreds = genderNet.forward()
    gender = genderList[genderPreds[0].argmax()]

    # Yaş tahmini yapın
    ageNet.setInput(blob)
    agePreds = ageNet.forward()
    age = ageList[agePreds[0].argmax()]

    # Sonuçları ekrana yazdırın
    label = "Gender: {}, Age: {}".format(gender, age)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# Çerçeveyi gösterin
cv2.imshow('Age Gender Demo', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Aynı modeli kendi kameramızda uygulamak için:

In [38]:
import cv2

# Yüz algılaması için CascadeClassifier'ı yükleyin
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Kamera veya görüntüyü açın
cap = cv2.VideoCapture(0)  # Eğer bir kamera kullanıyorsanız, 0 yerine kamera numarasını verin

while True:
    # Görüntüyü yakalayın
    ret, frame = cap.read()

    # Yakalanan görüntüyü gri tonlamalıya dönüştürün
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Yüz algılaması yapın
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Algılanan yüz bölgelerini işleyin
    for (x, y, w, h) in faces:
        # Algılanan yüzü çerçeve üzerine çizin
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Sonucu gösterin
    cv2.imshow('Face Detection', frame)

    # Çıkış için 'q' tuşuna basın
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Kamera veya görüntüyü serbest bırakın ve pencereyi kapatın
cap.release()
cv2.destroyAllWindows()

In [40]:
import cv2

# Yüz algılaması için CascadeClassifier'ı yükleyin
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Kamera veya görüntüyü açın
cap = cv2.VideoCapture(0)  # Eğer bir kamera kullanıyorsanız, 0 yerine kamera numarasını verin

# Yaş ve cinsiyet tahmini için model ve prototip dosyalarını yükleyin
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
genderNet = cv2.dnn.readNet(genderModel, genderProto)

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
ageNet = cv2.dnn.readNet(ageModel, ageProto)

ageList = ['(0 - 2)', '(4 - 6)', '(8 - 12)', '(15 - 20)', '(25 - 32)', '(38 - 43)', '(48 - 53)', '(60 - 100)']
genderList = ['Male', 'Female']

while True:
    # Görüntüyü yakalayın
    ret, frame = cap.read()

    # Yakalanan görüntüyü gri tonlamalıya dönüştürün
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Yüz algılaması yapın
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Algılanan yüz bölgelerini işleyin
    for (x, y, w, h) in faces:
        # Yüzü çerçeve üzerine çizin
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Yüz bölgesini kırpmak için işlem yapın
        face = frame[y:y + h, x:x + w]

        # Yüzü yaş ve cinsiyet tahmini için kullanın
        blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        # Cinsiyet tahmini yapın
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]

        # Yaş tahmini yapın
        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]

        # Tahminleri çerçeve üzerine ekleyin
        label = "{}, {}".format(gender, age)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    # Sonucu gösterin
    cv2.imshow('Age Gender Detection', frame)

    # Çıkış için 'q' tuşuna basın
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Kamera veya görüntüyü serbest bırakın ve pencereyi kapatın
cap.release()
cv2.destroyAllWindows()

### PyTorch Kullanarak Makine Öğrenimi ile Görüntü Tanıma

In [ ]:
# Burdan İtibaren Colabta yapılması daha iyi.

In [8]:
pip install torchvision

^C
Note: you may need to restart the kernel to use updated packages.


In [7]:
from torchvision import models
dir(models)

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
alexnet = models.AlexNet()

In [ ]:
from torchvision import transforms
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

Bu durumda, giriş görüntüsünü 256 × 256'ya ölçeklendirecek, görüntüyü merkez çevresinden 224 × 224'e kırpacak, tensöre dönüştürecek ve RGB bileşenlerini normalleştirecek bir ön işleme fonksiyonu tanımladık. (kırmızı, yeşil, mavi) böylece ortalamaları ve standart sapmaları tanımlamışlar.

In [ ]:
#Resnet101 yöntemini kullanarak artık 101 katmanlı bir evrişimli sinir ağı oluşturabiliyoruz.

In [ ]:
resnet = models.resnet101(pretrained=True)
resnet

In [ ]:
from google.colab import files
uploaded = files.upload()
from PIL import Image
img = Image.open("dog.png")
img

In [ ]:
img_t = preprocess(img)

In [ ]:
import torch
batch_t = torch.unsqueeze(img_t, 0)
resnet.eval()
out = resnet(batch_t)
out

In [ ]:
with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]
_, index = torch.max(out, 1)

percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()